# Multi-Qubit Gates - Complete Guide for Qiskit Certification

**Section 1: Quantum Operations - Part 2**

This notebook covers multi-qubit gates and entanglement - the foundation of quantum advantage!

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- Master CNOT (CX) gate and its role in entanglement
- Create and understand all four Bell states
- Work with CZ, SWAP, and Toffoli gates
- Understand phase kickback mechanism
- Apply multi-qubit gate patterns for exam success

### 🧠 Key Exam Mnemonic: "Control BEFORE Target"

**Remember**: `qc.cx(control, target)`
- Control is the FIRST parameter
- Target is the SECOND parameter
- `qc.cx(0, 1)`: q0 controls, q1 is flipped

**⚠️ EXAM TRAP**: Direction matters!
```python
qc.cx(0, 1) ≠ qc.cx(1, 0)
|10⟩ →CX(0,1)→ |11⟩  BUT |10⟩ →CX(1,0)→ |10⟩ (no change!)
```

---

## Setup

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator
from qiskit.visualization import plot_bloch_multivector, plot_state_city
import matplotlib.pyplot as plt

%matplotlib inline

print("✅ All imports successful!")

---

## 1. CNOT Gate (CX) - MOST IMPORTANT

**Controlled-NOT**: Flips target if control is |1⟩

Syntax: `qc.cx(control, target)` or `qc.cnot(control, target)`

Truth table:
- |00⟩ → |00⟩
- |01⟩ → |01⟩
- |10⟩ → |11⟩  (flips target)
- |11⟩ → |10⟩  (flips target)

In [ ]:
# CNOT Gate Basics
qc_cnot = QuantumCircuit(2)
qc_cnot.cx(0, 1)  # Control=qubit 0, Target=qubit 1

print("Circuit:")
display(qc_cnot.draw(output='mpl'))

print("\nRemember: cx(control, target)")
print("Top qubit (0) = control, Bottom qubit (1) = target")

In [ ]:
# CNOT Truth Table
print("CNOT Truth Table")
print("=" * 50)

basis_states = ['00', '01', '10', '11']
for state in basis_states:
    qc = QuantumCircuit(2)
    
    # Prepare input state
    if state[0] == '1':
        qc.x(0)
    if state[1] == '1':
        qc.x(1)
    
    # Apply CNOT
    qc.cx(0, 1)
    
    # Get output
    result = Statevector(qc)
    output = ''.join(['1' if abs(result.data[i]) > 0.5 else '0' 
                     for i in range(4) if abs(result.data[i]) > 0.5])
    
    print(f"|{state}⟩ → |{output}⟩")

In [ ]:
# Creating Entanglement with H + CNOT
qc_bell = QuantumCircuit(2)
qc_bell.h(0)
qc_bell.cx(0, 1)

state = Statevector(qc_bell)

print("Circuit:")
display(qc_bell.draw(output='mpl'))

print(f"\nState vector: {state.data}")
print(f"This is Bell state |Φ+⟩ = (|00⟩ + |11⟩)/√2")

probs = np.abs(state.data)**2
print(f"\nProbabilities:")
print(f"|00⟩: {probs[0]:.2f}")
print(f"|11⟩: {probs[3]:.2f}")

print("\nVisualization:")
plot_state_city(state)

**🎯 EXAM TIP**: 
- H + CNOT is THE most common pattern!
- CNOT creates entanglement from superposition
- Used in EVERY Bell state circuit

### ⚠️ EXAM TRAP: CNOT Direction Matters!

**Critical Mistake to Avoid**:
```python
# These are DIFFERENT operations!
qc.cx(0, 1)  # q0 controls q1
qc.cx(1, 0)  # q1 controls q0
```

**Example**:
- State: |10⟩
- CX(0,1): |10⟩ → |11⟩ (control=1, so flip target)
- CX(1,0): |10⟩ → |10⟩ (control=0, so NO flip)

**Exam Pattern**: "What's the output after CX(a,b) on state |ψ⟩?"
- Always check which qubit is control!
- Control qubit never changes
- Target flips only if control=|1⟩

---

---

## 2. CZ Gate (Controlled-Z)

Applies Z to target if control is |1⟩

**Key Property**: CZ is symmetric (control and target are interchangeable)

In [ ]:
# CZ Gate
qc_cz = QuantumCircuit(2)
qc_cz.cz(0, 1)

print("Circuit:")
display(qc_cz.draw(output='mpl'))

# Matrix representation
cz_matrix = Operator(qc_cz).data
print("\nCZ Matrix:")
print(cz_matrix)

print("\nCZ Effect on Basis States:")
print("|00⟩ → |00⟩")
print("|01⟩ → |01⟩")
print("|10⟩ → |10⟩")
print("|11⟩ → -|11⟩  (phase flip)")

In [ ]:
# CZ ≡ H-CNOT-H Sandwich
qc_equiv = QuantumCircuit(2)
qc_equiv.h(1)
qc_equiv.cx(0, 1)
qc_equiv.h(1)

equiv_matrix = Operator(qc_equiv).data

print("H-CNOT-H circuit:")
display(qc_equiv.draw(output='mpl'))

print(f"\nMatrices equal: {np.allclose(cz_matrix, equiv_matrix)} ✓")

**🎯 EXAM TIP**: 
- CZ is symmetric: cz(0,1) = cz(1,0)
- Unlike CNOT which is not symmetric

---

## 3. SWAP Gate

Swaps two qubits: |01⟩ ↔ |10⟩

Can be decomposed as 3 CNOTs

In [ ]:
# SWAP Gate
qc_swap = QuantumCircuit(2)
qc_swap.swap(0, 1)

print("Circuit:")
display(qc_swap.draw(output='mpl'))

In [ ]:
# Test on |01⟩
qc_test = QuantumCircuit(2)
qc_test.x(1)  # Prepare |01⟩
qc_test.swap(0, 1)

state = Statevector(qc_test)
print(f"Input: |01⟩")
print(f"Output: {state.data}")
print(f"Result: |10⟩ (qubits swapped)")

In [ ]:
# SWAP Decomposition (3 CNOTs)
qc_decomp = QuantumCircuit(2)
qc_decomp.cx(0, 1)
qc_decomp.cx(1, 0)
qc_decomp.cx(0, 1)

print("SWAP = 3 CNOTs:")
display(qc_decomp.draw(output='mpl'))

decomp_matrix = Operator(qc_decomp).data
swap_matrix = Operator(qc_swap).data
print(f"\nMatrices equal: {np.allclose(swap_matrix, decomp_matrix)} ✓")

**🎯 EXAM TIP**: 
- SWAP exchanges qubit states
- Can be decomposed into 3 CNOTs

---

## 4. Toffoli Gate (CCX)

**Controlled-Controlled-NOT**

Flips target if BOTH controls are |1⟩

Syntax: `qc.ccx(control1, control2, target)`

In [ ]:
# Toffoli Gate
qc_ccx = QuantumCircuit(3)
qc_ccx.ccx(0, 1, 2)  # Controls: 0,1 | Target: 2

print("Circuit:")
display(qc_ccx.draw(output='mpl'))
print("\nSyntax: ccx(control1, control2, target)")

In [ ]:
# Toffoli Truth Table
print("Toffoli Truth Table (Target flips only if both controls=1)")
print("=" * 50)
print("Input → Output")
print("=" * 50)

test_states = ['000', '001', '010', '011', '100', '101', '110', '111']

for state in test_states:
    qc = QuantumCircuit(3)
    
    # Prepare input
    for i, bit in enumerate(state):
        if bit == '1':
            qc.x(i)
    
    # Apply Toffoli
    qc.ccx(0, 1, 2)
    
    # Get output state
    result = Statevector(qc)
    for i, amp in enumerate(result.data):
        if abs(amp) > 0.9:
            output = format(i, '03b')
            break
    
    marker = " ← Target flipped" if state[:2] == '11' else ""
    print(f"|{state}⟩ → |{output}⟩{marker}")

In [ ]:
# Toffoli as AND Gate
qc_and = QuantumCircuit(3)
qc_and.x(0)
qc_and.x(1)
qc_and.ccx(0, 1, 2)

print("Toffoli implements AND:")
display(qc_and.draw(output='mpl'))

print(f"\n|110⟩ → |111⟩")
print(f"Target qubit computes: q0 AND q1")

**🎯 EXAM TIP**: 
- Toffoli has 2 controls, 1 target
- Flips target ONLY if both controls are |1⟩
- Classical: Implements AND operation

---

## 5. Bell States - MEMORIZE ALL 4!

**⚠️ GUARANTEED EXAM QUESTIONS! ⚠️**

Bell states are maximally entangled 2-qubit states:

- |Φ+⟩ = (|00⟩ + |11⟩)/√2
- |Φ-⟩ = (|00⟩ - |11⟩)/√2
- |Ψ+⟩ = (|01⟩ + |10⟩)/√2
- |Ψ-⟩ = (|01⟩ - |10⟩)/√2

In [ ]:
# Bell state |Φ+⟩
qc_phi_plus = QuantumCircuit(2)
qc_phi_plus.h(0)
qc_phi_plus.cx(0, 1)

state_phi_plus = Statevector(qc_phi_plus)

print("|Φ+⟩ = (|00⟩ + |11⟩)/√2")
print("=" * 50)
display(qc_phi_plus.draw(output='mpl'))
print(f"State vector: {state_phi_plus.data}")
print(f"Recipe: H(0) + CNOT(0,1)")

plot_state_city(state_phi_plus)

### State Evolution: |Ψ-⟩ Step-by-Step

**Circuit**:
```
q0: ──H──Z──●──
            │
q1: ────X───X──
```

**Step-by-Step Evolution**:
```
Step 1: Initial state
|ψ₀⟩ = |00⟩

Step 2: After H(q0)
|ψ₁⟩ = (|00⟩+|10⟩)/√2

Step 3: After Z(q0)
|ψ₂⟩ = (|00⟩-|10⟩)/√2

Step 4: After X(q1)
|ψ₃⟩ = I⊗X (|00⟩-|10⟩)/√2
     = (|01⟩-|11⟩)/√2

Step 5: After CNOT(q0,q1)
|ψ₄⟩ = CNOT (|01⟩-|11⟩)/√2
     = (|01⟩-|10⟩)/√2
     = |Ψ-⟩ ✓

Key difference from |Ψ+⟩:
- Negative phase between |01⟩ and |10⟩
```

---

In [ ]:
# Bell state |Φ-⟩
qc_phi_minus = QuantumCircuit(2)
qc_phi_minus.h(0)
qc_phi_minus.z(0)  # Add phase
qc_phi_minus.cx(0, 1)

state_phi_minus = Statevector(qc_phi_minus)

print("|Φ-⟩ = (|00⟩ - |11⟩)/√2")
print("=" * 50)
display(qc_phi_minus.draw(output='mpl'))
print(f"State vector: {state_phi_minus.data}")
print(f"Recipe: H(0) + Z(0) + CNOT(0,1)")

plot_state_city(state_phi_minus)

### State Evolution: |Ψ+⟩ Step-by-Step

**Circuit**:
```
q0: ──H─────●──
            │
q1: ────X───X──
```

**Step-by-Step Evolution**:
```
Step 1: Initial state
|ψ₀⟩ = |00⟩

Step 2: After H(q0)
|ψ₁⟩ = (|00⟩+|10⟩)/√2

Step 3: After X(q1)
|ψ₂⟩ = I⊗X (|00⟩+|10⟩)/√2
     = (|01⟩+|11⟩)/√2
     (Flip q1: |0⟩→|1⟩)

Step 4: After CNOT(q0,q1)
|ψ₃⟩ = CNOT (|01⟩+|11⟩)/√2
     = (|01⟩+|10⟩)/√2
     = |Ψ+⟩ ✓

Key difference from |Φ+⟩:
- Swapped: |00⟩,|11⟩ → |01⟩,|10⟩
```

---

In [ ]:
# Bell state |Ψ+⟩
qc_psi_plus = QuantumCircuit(2)
qc_psi_plus.h(0)
qc_psi_plus.x(1)  # Flip target first
qc_psi_plus.cx(0, 1)

state_psi_plus = Statevector(qc_psi_plus)

print("|Ψ+⟩ = (|01⟩ + |10⟩)/√2")
print("=" * 50)
display(qc_psi_plus.draw(output='mpl'))
print(f"State vector: {state_psi_plus.data}")
print(f"Recipe: H(0) + X(1) + CNOT(0,1)")

plot_state_city(state_psi_plus)

### State Evolution: |Φ+⟩ Step-by-Step

**Circuit**:
```
q0: ──H──●──
         │
q1: ─────X──
```

**Step-by-Step Evolution**:
```
Step 1: Initial state
|ψ₀⟩ = |00⟩

Step 2: After H(q0)
|ψ₁⟩ = H⊗I |00⟩
     = (|0⟩+|1⟩)/√2 ⊗ |0⟩
     = (|00⟩+|10⟩)/√2

Step 3: After CNOT(q0,q1)
|ψ₂⟩ = CNOT (|00⟩+|10⟩)/√2
     = (|00⟩+|11⟩)/√2
     = |Φ+⟩ ✓

Measurement probabilities:
- P(|00⟩) = 1/2 = 50%
- P(|11⟩) = 1/2 = 50%
- Perfect correlation!
```

---

In [ ]:
# Bell state |Ψ-⟩
qc_psi_minus = QuantumCircuit(2)
qc_psi_minus.h(0)
qc_psi_minus.z(0)  # Add phase
qc_psi_minus.x(1)  # Flip target
qc_psi_minus.cx(0, 1)

state_psi_minus = Statevector(qc_psi_minus)

print("|Ψ-⟩ = (|01⟩ - |10⟩)/√2")
print("=" * 50)
display(qc_psi_minus.draw(output='mpl'))
print(f"State vector: {state_psi_minus.data}")
print(f"Recipe: H(0) + Z(0) + X(1) + CNOT(0,1)")

plot_state_city(state_psi_minus)

### State Evolution: |Φ-⟩ Step-by-Step

**Circuit**:
```
q0: ──H──Z──●──
            │
q1: ────────X──
```

**Step-by-Step Evolution**:
```
Step 1: Initial state
|ψ₀⟩ = |00⟩

Step 2: After H(q0)
|ψ₁⟩ = (|00⟩+|10⟩)/√2

Step 3: After Z(q0)
|ψ₂⟩ = Z⊗I (|00⟩+|10⟩)/√2
     = (|00⟩-|10⟩)/√2
     (Z adds minus to |1⟩ component)

Step 4: After CNOT(q0,q1)
|ψ₃⟩ = CNOT (|00⟩-|10⟩)/√2
     = (|00⟩-|11⟩)/√2
     = |Φ-⟩ ✓

Key difference from |Φ+⟩: 
- Negative phase between |00⟩ and |11⟩
```

---

In [ ]:
# Bell States Summary Table
print("BELL STATES SUMMARY - MEMORIZE THIS!")
print("=" * 70)
print("State  | Expression              | Gates")
print("=" * 70)
print("|Φ+⟩   | (|00⟩ + |11⟩)/√2       | H(0) + CNOT(0,1)")
print("|Φ-⟩   | (|00⟩ - |11⟩)/√2       | H(0) + Z(0) + CNOT(0,1)")
print("|Ψ+⟩   | (|01⟩ + |10⟩)/√2       | H(0) + X(1) + CNOT(0,1)")
print("|Ψ-⟩   | (|01⟩ - |10⟩)/√2       | H(0) + Z(0) + X(1) + CNOT(0,1)")
print("=" * 70)

**🎯 EXAM TIP**: 
- H + CNOT creates |Φ+⟩
- Add Z(0) for negative phase: Φ+ → Φ-
- Add X(1) to swap: Φ → Ψ
- **Memorize all 4 circuits!**

---

## 6. GHZ States - Multi-qubit Entanglement

**GHZ = Greenberger-Horne-Zeilinger state**

Pattern: H on first qubit + CNOT chain

In [ ]:
# Demonstrate Phase Kickback
qc_kickback = QuantumCircuit(2)
qc_kickback.h(0)        # Control in superposition
qc_kickback.x(1)        # Prepare target |1⟩
qc_kickback.h(1)        # Target becomes |-⟩
qc_kickback.cx(0, 1)    # Phase kickback!
qc_kickback.h(1)        # Return target to computational basis

state_kickback = Statevector(qc_kickback)

print("Phase Kickback Circuit:")
display(qc_kickback.draw(output='mpl'))

print(f"\nFinal state: {state_kickback.data}")
print("\nNotice: Phase has been transferred to control qubit!")


---

## 5C. Phase Kickback - Advanced Concept

**Critical for Deutsch-Jozsa, Grover's Algorithm!**

### What is Phase Kickback?

When a controlled operation acts on a target in superposition, the phase "kicks back" to the control qubit!

### Classic Example:
```python
qc.h(0)        # Control in |+⟩
qc.x(1)        # Target to |1⟩
qc.h(1)        # Target in |-⟩
qc.cx(0, 1)    # Phase kickback happens here!
```

**What happens**:
- Target in |-⟩ = (|0⟩-|1⟩)/√2
- CNOT on |-⟩ adds phase to control
- Control picks up the phase difference!

### Why it matters for exams:
- Appears in Deutsch-Jozsa algorithm
- Used in Grover's search
- Enables quantum parallelism
- **Key insight**: Phase information transfers to control

---

---

## 5B. Entanglement Patterns - EXAM ESSENTIALS

### Pattern 1: Bell State Creation
```python
qc.h(0)      # Create superposition
qc.cx(0, 1)  # Create entanglement
# Result: (|00⟩+|11⟩)/√2
```

**Why it works**:
- H creates superposition: (|0⟩+|1⟩)/√2 ⊗ |0⟩ = (|00⟩+|10⟩)/√2
- CNOT correlates qubits: |00⟩ stays, |10⟩→|11⟩
- Outcome: Perfect correlation!

### Pattern 2: Multi-Qubit Superposition
```python
# Create all 2^n states equally
for i in range(n):
    qc.h(i)
# Result: (|00...0⟩ + |00...1⟩ + ... + |11...1⟩) / √(2^n)
```

**Exam Fact**: H^⊗n creates superposition of ALL 2^n basis states

---

In [ ]:
# 3-qubit GHZ
qc_ghz3 = QuantumCircuit(3)
qc_ghz3.h(0)
qc_ghz3.cx(0, 1)
qc_ghz3.cx(0, 2)

state_ghz3 = Statevector(qc_ghz3)

print("3-Qubit GHZ State")
print("=" * 50)
display(qc_ghz3.draw(output='mpl'))
print(f"\nState vector: {state_ghz3.data}")
print(f"|GHZ⟩ = (|000⟩ + |111⟩)/√2")

plot_state_city(state_ghz3)

In [ ]:
# 4-qubit GHZ
qc_ghz4 = QuantumCircuit(4)
qc_ghz4.h(0)
qc_ghz4.cx(0, 1)
qc_ghz4.cx(0, 2)
qc_ghz4.cx(0, 3)

print("4-Qubit GHZ State")
print("=" * 50)
display(qc_ghz4.draw(output='mpl'))
print(f"|GHZ⟩₄ = (|0000⟩ + |1111⟩)/√2")

**🎯 EXAM TIP**: 
- GHZ extends Bell states to n qubits
- Pattern: H + CNOT cascade
- All qubits become correlated

---

## 7. Multi-Qubit Gates Comparison

In [ ]:
print("MULTI-QUBIT GATES COMPARISON")
print("=" * 70)
print("Gate   | Qubits | Description                    | Key Property")
print("=" * 70)
print("CNOT   | 2      | Controlled-NOT                 | Creates entanglement")
print("CZ     | 2      | Controlled-Z                   | Symmetric")
print("SWAP   | 2      | Exchange qubits                | 3 CNOTs")
print("CCX    | 3      | Toffoli (2 controls)           | Universal")
print("Bell   | 2      | 4 maximally entangled states   | H + CNOT base")
print("GHZ    | n      | Multi-qubit entanglement       | H + CNOT chain")
print("=" * 70)

---

## ✅ Key Takeaways

1. **CNOT** creates entanglement - MOST IMPORTANT
2. **All 4 Bell states** use H + CNOT pattern
3. **CZ is symmetric**, CNOT is not
4. **SWAP** exchanges qubits (3 CNOTs)
5. **Toffoli (CCX)** has 2 controls
6. **GHZ** extends entanglement to n qubits

**🎓 Practice Bell states until you can draw them instantly!**